## 에이전트에 메모리 추가하기

이 과정에서는 에이전트에 메모리를 추가하는 방법을 보여줍니다. 이를 위해 ContextProvider을 구현하고 에이전트에 연결합니다.

> 중요  
>   
> 모든 에이전트 유형이 ContextProvider를 지원하지는 않습니다. 이 과정에서는 ContextProvider를 지원하는 ChatAgent를 사용합니다.

#### ContextProvider 생성
ContextProvider는 ChatAgent의 AgentThread와 연결할 수 있는, 상속 가능한 추상 클래스입니다. 이를 통해 다음과 같은 작업을 수행할 수 있습니다.

- 에이전트가 내부 추론 서비스를 호출하기 전,후에 사용자 지정 로직을 실행합니다.
- 에이전트가 내부 추론 서비스를 호출하기 전에 추가적인 컨텍스트를 제공합니다.
- 에이전트에게 제공되거나 에이전트가 생성하는 모든 메시지를 검사합니다.

##### 호출 전,후 이벤트
ContextProvider 클래스에는 에이전트가 내부 추론 서비스를 호출하기 전, 후에 사용자 지정 로직을 실행하기 위해 재정의할 수 있는 2개의 메서드가 있습니다.
invoking - 에이전트가 내부 추론 서비스를 호출하기 전에 호출됩니다. context 객체를 반환하여 에이전트에 추가적인 컨텍스트를 제공할 수 있습니다. 이 컨텍스트는 기본 서비스를 호출하기 전에 에이전트의 기존 컨텍스트와 병합됩니다. 요청에 추가할 지침(instruction), 도구(tool) 및 메시지를 제공할 수 있습니다.

invoked- 에이전트가 내부 추론 서비스로부터 응답을 받은 후에 호출됩니다. 요청 및 응답 메시지를 검사하고 컨텍스트 공급자의 상태를 업데이트할 수 있습니다.

#### 직렬화
ContextProvider 인스턴스는 스레드가 생성될 때와 쓰레드가 직렬화된 상태에서 재개(Resume)될 때 생성되고 AgentThread에 연결됩니다 .

해당 ContextProvider 인스턴스는 에이전트 호출 간에 유지되어야 하는 자체 상태(state)를 가질 수 있습니다. 예를 들어, 사용자에 대한 정보를 기억하는 메모리 구성 요소는 상태(state)의 일부로 메모리들을 포함할 수 있습니다.

스레드가 계속 유지되게 하려면, 클래스에 직렬화 기능을 구현해야 합니다 ContextProvider. 또한 스레드를 재개할 때 직렬화된 데이터에서 상태를 복원할 수 있는 생성자를 제공해야 합니다.

#### 샘플 ContextProvider 구현 예시

다음은 사용자 지정 메모리 구성 요소의 예입니다. 이 구성 요소는 사용자의 이름과 나이를 기억하고 각 호출 전에 에이전트에 제공합니다. 먼저, 메모리를 저장할 모델 클래스를 만듭니다.


In [1]:
from pydantic import BaseModel

class UserInfo(BaseModel):
    name: str | None = None
    age: int | None = None

그런 다음, ContextProvider를 구현하여 메모리 관리할 수 있습니다. UserInfoMemory 클래스는 다음과 같은 동작을 포함합니다.

1. 채팅 클라이언트를 사용하여 매 실행이 끝날 때 새 메시지가 스레드에 추가되면 사용자 메시지에서 사용자의 이름과 나이를 찾습니다.
2. 이는 각 호출 전에 에이전트에게 현재 모든 메모리 정보를 제공합니다.
3. 만약 가용한 메모리가 없다면, 이는 에이전트가 사용자에게 누락된 정보를 요청하고 정보가 제공될 때까지 어떠한 질문에도 답변하지 않도록 합니다.
4. 또한 스레드 상태의 일부로 메모리를 영구 저장할 수 있도록 직렬화를 구현합니다.


In [10]:
from collections.abc import MutableSequence, Sequence
from typing import Any

from agent_framework import ContextProvider, Context, ChatAgent, ChatClientProtocol, ChatMessage, ChatOptions


class UserInfoMemory(ContextProvider):
    def __init__(self, chat_client: ChatClientProtocol, user_info: UserInfo | None = None, **kwargs: Any):
        """Create the memory.

        If you pass in kwargs, they will be attempted to be used to create a UserInfo object.
        """
        self._chat_client = chat_client
        if user_info:
            self.user_info = user_info
        elif kwargs:
            self.user_info = UserInfo.model_validate(kwargs)
        else:
            self.user_info = UserInfo()

    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs: Any,
    ) -> None:
        """Extract user information from messages after each agent call."""
        # Ensure request_messages is a list
        messages_list = [request_messages] if isinstance(request_messages, ChatMessage) else list(request_messages)

        # Check if we need to extract user info from user messages
        user_messages = [msg for msg in messages_list if msg.role.value == "user"]

        if (self.user_info.name is None or self.user_info.age is None) and user_messages:
            try:
                # Use the chat client to extract structured information
                result = await self._chat_client.get_response(
                    messages=messages_list,
                    chat_options=ChatOptions(
                        instructions=(
                            "Extract the user's name and age from the message if present. "
                            "If not present return nulls."
                        ),
                        response_format=UserInfo,
                    ),
                )

                # Update user info with extracted data
                if result.value and isinstance(result.value, UserInfo):
                    if self.user_info.name is None and result.value.name:
                        self.user_info.name = result.value.name
                    if self.user_info.age is None and result.value.age:
                        self.user_info.age = result.value.age

            except Exception:
                pass  # Failed to extract, continue without updating

    async def invoking(self, messages: ChatMessage | MutableSequence[ChatMessage], **kwargs: Any) -> Context:
        """Provide user information context before each agent call."""
        instructions: list[str] = []

        if self.user_info.name is None:
            instructions.append(
                "Ask the user for their name and politely decline to answer any questions until they provide it."
            )
        else:
            instructions.append(f"The user's name is {self.user_info.name}.")

        if self.user_info.age is None:
            instructions.append(
                "Ask the user for their age and politely decline to answer any questions until they provide it."
            )
        else:
            instructions.append(f"The user's age is {self.user_info.age}.")

        # Return context with additional instructions
        return Context(instructions=" ".join(instructions))

    def serialize(self) -> str:
        """Serialize the user info for thread persistence."""
        return self.user_info.model_dump_json()

#### 에이전트와 함께 ContextProvider 사용하기

사용자 지정 ContextProvider를 사용하려면, 에이전트를 생성할 때 인스턴스화된 ContextProvider 객체를 제공해야 합니다.
그리고, ChatAgent를 생성할 때 메모리 구성 요소를 에이전트에 연결하기 위해 context_providers 매개변수를 제공해야 합니다.


In [18]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일 로드

True

In [ ]:
import asyncio
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

async def main():
    async with AzureCliCredential() as credential:
        async with AzureAIAgentClient(credential=credential) as chat_client:
            # Create the memory provider
            memory_provider = UserInfoMemory(chat_client)

            # Create the agent with memory - context_providers를 agent 생성 시 전달
            async with ChatAgent(
                chat_client=chat_client,
                instructions="You are a friendly assistant. Always address the user by their name.",
                context_providers=memory_provider,
            ) as agent:
                # Create a new thread for the conversation
                thread = agent.get_new_thread()

                result2 = await agent.run("My name is Ruaidhrí", thread=thread)
                print("결과 : " + result2.text)

                result3 = await agent.run("I am 20 years old", thread=thread)
                print("결과 : " + result3.text)

                # Access the memory component via the thread's context_providers attribute and inspect the memories
                if thread.context_provider:
                    user_info_memory = thread.context_provider.providers[0]
                    if isinstance(user_info_memory, UserInfoMemory):
                        print()
                        print(f"MEMORY - User Name: {user_info_memory.user_info.name}")
                        print(f"MEMORY - User Age: {user_info_memory.user_info.age}")

if __name__ == "__main__":
    await main()

Hello! The square root of 9 is **3**.

(Mathematically, √9 = 3.)

Note: Technically, both 3 and -3 squared will give 9, so the square roots of 9 are **3 and -3**. However, the *principal* or *positive* square root is 3.
Hello, Ruaidhrí! That's a beautiful name. How can I help you today? 😊
Nice to meet you, Ruaidhrí! At 20, you have so many exciting opportunities ahead. Is there something specific you'd like to talk about or ask? 😊
